In [ ]:
import numpy as np
from numpy import pi
from scipy.interpolate import CubicSpline
from integrate.lowStorageRungeKuttaMethods import LSRK3
from derivative.spectralMethods import spectralMethod1
from utils.grid import wallNormalGrid
from scipy.sparse.linalg import LinearOperator, gmres
from utils.wallLaw import wallLow

In [ ]:
class channel2(object):
    Re_tau = NotImplemented
    nx = NotImplemented
    ny = NotImplemented
    lx = NotImplemented
    nu = NotImplemented
    delta = NotImplemented
    eta = NotImplemented
    dt = NotImplemented
    def __init__(self):
        self.x = np.linspace(0, self.lx, self.nx, endpoint=False)
        self.y = wallNormalGrid(self.ny, self.eta, self.delta)
        self.X, self.Y = np.meshgrid(self.x, self.y)
        self.yPlus = (self.delta - np.abs(self.y)) * self.Re_tau
        self.u_tau = self.Re_tau * self.nu / self.delta
        self.dx = self.lx / self.nx
        self.integrate = LSRK3()
        self.spectral = spectralMethod1(n=self.nx, d=self.dx)
        self.force = wallLow(self.Re_tau) ** -2
        self.p_shape = (self.ny, self.nx)
        self.A_shape = (self.nx*self.ny, self.nx*self.ny)
        self.exitCode = 0
        self.A = LinearOperator(self.A_shape, matvec=self.lhs2)

    def rhs1(self, t, u):
        u_interp = CubicSpline(self.y, u, axis=-2, bc_type='not-a-knot')
        return ( - u[0] * self.spectral.diff_phys(u)
                 - u[1] * u_interp(self.y, nu=1)
                 + self.nu * (self.spectral.diff_phys(u, order=2)
                            + u_interp(self.y, nu=2))
                 + self.force )

    def lhs2(self, p_flatten):
        p = np.reshape(p_flatten, self.p_shape)
        p_interp = CubicSpline(self.y, p, axis=-2, bc_type='clamped')
        p = self.spectral.diff_phys(p, order=2) + p_interp(self.y, nu=2)
        return p.flatten()

    def rhs2(self, u):
        u_interp = CubicSpline(self.y, u[1], axis=-2, bc_type='not-a-knot')
        return (self.spectral.diff_phys(u[0])
                + u_interp(self.y, nu=1)) / self.dt

    def rhs3(self):
        p_interp = CubicSpline(self.y, self.p, axis=-2, bc_type='clamped')
        return np.stack((self.spectral.diff_phys(self.p, order=1), p_interp(self.y, nu=1)))

    def step(self):
        u = self.integrate.step(fun=self.rhs1, u=self.u, t0=0, dt=self.dt)
        self.u[:,(0,-1),:] = 0.
        self.p, self.exitCode = gmres(A=self.A, b=self.rhs2(u).flatten(), x0=self.p.flatten())
        self.p = np.reshape(self.p, self.p_shape)
        self.u -= self.dt * self.rhs3()
        self.u[:,(0,-1),:] = 0.

    def solve(self, u0, t0, ts_span, p0=None):
        self.u = u0
        self.t = t0
        if p0 == None:
            self.p = np.zeros(self.p_shape)
        else:
            self.p = p0
        for ts in range(ts_span[0], ts_span[1]+1):
            print(ts)
            self.step()
            self.t = t0 + self.dt*ts

In [ ]:
class channel2_100(channel2):
    Re_tau = 100
    nx = 168
    ny = 127
    lx = 2. * pi
    nu = 1.
    delta = 1.
    eta = 0.94
    dt = 0.004


class channel2_180(channel2):
    Re_tau = 180
    nx = 256
    ny = 191
    lx = 2. * pi
    nu = 1.
    delta = 1.
    eta = 0.95
    dt = 0.002


class channel2_400(channel2):
    Re_tau = 400
    nx = 384
    ny = 255
    lx = 2. * pi
    nu = 1.
    delta = 1.
    eta = 0.97
    dt = 0.001

In [ ]:
tcf = channel2_100()
v0 = np.zeros((tcf.ny, tcf.nx))
u0 = np.expand_dims(wallLow(tcf.yPlus) * tcf.u_tau, axis=1) * np.ones_like(v0)
U0 = np.stack((u0, v0))
t0 = 0
ts_span = (0, 10)
# tcf.solve(U0, t0, ts_span)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.contourf(tcf.X, tcf.Y, u0, cmap="coolwarm")
plt.show()

In [ ]:
Ret = 400
nu = 1 / Ret
delta = 1
u_tau  = Ret * nu / delta
print(u_tau)
u_mc = 1
u_mc_p = 20.5
u_tau = u_mc / u_mc_p
print(u_tau)
Rec = u_mc * delta / nu
print(Rec)